## Setup


In [1]:
%%capture
!pip install wandb --upgrade
!pip install tensorflow-addons

In [241]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.layers import Embedding, SimpleRNNCell, GRUCell, Dense, LSTMCell
from tensorflow.keras import Input
import pandas as pd
from numpy import argmax
from math import log
import pprint
import math
import wandb
import os
import io
from wandb.keras import WandbCallback

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api")

#wandb.login(key=wandb_api)
! wandb login $wandb_api

os.environ["WANDB_SILENT"] = "true"
#wandb.login()

## Download the data


## Configuration


In [3]:
!wget  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'
train_file_path = "./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_file_path= "./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"
test_file_path  = "./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"

--2022-06-19 11:47:34--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 172.217.193.128, 142.250.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   204MB/s    in 9.0s    

2022-06-19 11:47:43 (214 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [8]:
batch_size = 256  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = train_file_path


## Prepare the data


In [287]:
def processData(filename,input_chars=set(),target_chars=set()):
  input=[]
  target=[]
  with open(filename, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:len(lines) -1]:
      t_text,i_text, attestation = line.split("\t")
       # We use "\t" as the "start sequence" character and "\n" as "end sequence" character for the target text.
      input.append(i_text)
      target.append("\t"+t_text+"\n")
      for char in i_text:
        if char not in input_chars:
            input_chars.add(char)
      for char in t_text:
        if char not in target_chars:
            target_chars.add(char)
  target_chars.add("\t")
  target_chars.add("\n")

  input_chars = sorted(list(input_chars))
  target_chars = sorted(list(target_chars))
  num_encoder_tokens = len(input_chars)
  num_decoder_tokens = len(target_chars)
  max_encoder_seq_length = max([len(txt) for txt in input])
  max_decoder_seq_length = max([len(txt) for txt in target])
  return input,target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length     

In [288]:
# Vectorize the data.
input,target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length=processData(train_file_path)
print("Number of samples:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 44204
Number of unique input tokens: 26
Number of unique output tokens: 65
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [297]:
# Vectorize the data.
# Vectorize the data.
validation_input,validation_target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, validation_max_encoder_seq_length, validation_max_decoder_seq_length=processData(val_file_path,set(input_chars),set(target_chars))

print("Number of validation samples:", len(validation_input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("validation Max sequence length for inputs:", validation_max_encoder_seq_length)
print("validation Max sequence length for outputs:", validation_max_decoder_seq_length)

Number of validation samples: 4502
Number of unique input tokens: 26
Number of unique output tokens: 65
validation Max sequence length for inputs: 16
validation Max sequence length for outputs: 17


In [263]:
# Vectorize the data.
test_input,test_target,test_input_chars,test_target_chars,test_num_encoder_tokens,test_num_decoder_tokens, test_max_encoder_seq_length, test_max_decoder_seq_length=processData(test_file_path)
print("Number of validation samples:", len(test_input))
print("Test Max sequence length for inputs:", test_max_encoder_seq_length)
print("Test Max sequence length for outputs:", test_max_decoder_seq_length)

Number of validation samples: 4358
Test Max sequence length for inputs: 18
Test Max sequence length for outputs: 16


In [229]:
input_token = dict([(char, i) for i, char in enumerate(input_chars)])
target_token = dict([(char, i) for i, char in enumerate(target_chars)])

reverse_input_token = dict((i, char) for char, i in input_token.items())
reverse_target_token = dict((i, char) for char, i in target_token.items())


encoder_input_data = np.zeros(
    (len(input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
validation_encoder_input_data=np.zeros(
    (len(validation_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
test_encoder_input_data=np.zeros(
    (len(test_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
validation_decoder_input_data =np.zeros(
    (len(validation_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
validation_decoder_target_data = np.zeros(
    (len(validation_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input, target)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token[char]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token[char]] = 1.0
# for validation data
for i, (validation_input_text, validation_target_text) in enumerate(zip(validation_input, validation_target)):
    for t, char in enumerate(validation_input_text):
        validation_encoder_input_data[i, t, input_token[char]] = 1.0
    for t, char in enumerate(validation_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        validation_decoder_input_data[i, t, target_token[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            validation_decoder_target_data[i, t - 1, target_token[char]] = 1.0

# for test data
for i, (test_input_text, test_target_text) in enumerate(zip(test_input, test_target)):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token[char]] = 1.0

In [299]:
enc_input_data = np.zeros(
    (len(input), max_encoder_seq_length), dtype="float32"
)
dec_input_data = np.zeros(
    (len(input), max_decoder_seq_length), dtype="float32"
)
dec_target_data = np.zeros(
    (len(input), max_decoder_seq_length,num_decoder_tokens), dtype="float32"
)
#Decoder Target Sequences are Padded to a maximum length of max_decoder SeqLen characters with a vocabulary of sizeofTeluguVocab different characters. 
for i, (input_text, target_text) in enumerate(zip(input, target)):
    for t, char in enumerate(input_text):
        enc_input_data[i, t] = input_token[char]
    #enc_input_data[i, t + 1 :] = input_token[" "]

    for t, char in enumerate(target_text):
        # dec_target_data is ahead of dec_input_data by one timestep
        dec_input_data[i, t] = target_token[char]
        if t > 0:
            # dec_target_data will not include the start character.
            dec_target_data[i, t - 1, target_token[char]] = 1.0
    #dec_input_data[i, t + 1: ] = target_token[" "]
    #dec_target_data[i, t:, target_token[" "]] = 1.0
    
val_enc_input_data = np.zeros(
    (len(validation_input), validation_max_encoder_seq_length), dtype="float32"
)
val_dec_input_data = np.zeros(
    (len(validation_input), validation_max_decoder_seq_length), dtype="float32"
)
val_dec_target_data = np.zeros(
    (len(validation_input), validation_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(validation_input,validation_target)):
    for t, char in enumerate(input_text):
        # Enumerate() method adds a counter to an iterable and returns it in a form of enumerating object. 
        # This enumerated object can then be used directly for loops or converted into a list of tuples using the list() method.
        val_enc_input_data[i, t] = input_token[char]
    #val_enc_input_data[i, t + 1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        val_dec_input_data[i, t] = target_token[char]
        if t > 0:
            # dec_target_data will be ahead by one timestep
            # and will not include the start character.
            val_dec_target_data[i, t - 1, target_token[char]] = 1.0
    #val_dec_input_data[i, t + 1: ] = target_token_index[" "]

## Build the model


In [248]:
class Seq2seq(tf.keras.Model):
  def __init__(self, num_encoder_tokens, num_decoder_tokens,embedding_dim,num_of_layers,unit_type, dropout , recurrent_dropout):
    super().__init__()
    self.encoder_inputs = Input(shape = (None,), name = "Input_layer_1")
    self.decoder_inputs = keras.Input(shape=(None,), name = "Input_layer_2")
    self.num_encoder_tokens = num_encoder_tokens
    self.embedding_dim = embedding_dim
    self.dropout = dropout
    self.recurrent_dropout = recurrent_dropout
    self.num_decoder_tokens = num_decoder_tokens
    self.num_of_encoder_layer  =num_of_layers
    self.num_of_decoder_layer =num_of_layers
    self.type_encoder_unit =unit_type 
    self.type_decoder_unit =unit_type
    self.train_step()
    self.build_model()

  def get_embedding_layer(self, num_encoder_tokens, embedding_dim,  name):
    return Embedding(num_encoder_tokens, embedding_dim, mask_zero = True, name =name )

  def get_cell(self, cell_type = "lstm", num_of_cell = 1, name = None):
    #print(cell_type)
    if cell_type == "lstm":
      return LSTMCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout, )
    elif cell_type == "rnn":
      return SimpleRNNCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    elif cell_type =="gru":
      return GRUCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    else:
      print(f"Invalid cell type: {cell_type}")
  def get_encoder(self,latent_dim, cell_type = "lstm", num_of_layer = 1, name = None ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim) for i in range(num_of_layer)],), return_sequences=True, return_state=True, name = name)

  def get_decoder(self,latent_dim ,cell_type = "lstm", num_of_layer = 1, name = None ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim,) for i in range(num_of_layer)]), return_sequences=True, return_state=True)

  def get_dense_layer(self, num_decoder_token, activation = "softmax"):
    return Dense(num_decoder_tokens, activation= activation)

  def train_step(self):
    self.embedding_layer = self.get_embedding_layer( self.num_encoder_tokens, self.embedding_dim ,name = "encoder_embedding")
    self.embedding_results = self.embedding_layer(self.encoder_inputs)
    print(self.embedding_results.shape)
    self.encoder = self.get_encoder( self.embedding_dim,self.type_encoder_unit, self.num_of_encoder_layer , name ="encoder" )
    encoder_results = self.encoder(self.embedding_results)

    self.encoder_outputs, self.encoder_states = encoder_results[0], encoder_results[1:]

    self.embedding_layer2 = self.get_embedding_layer( self.num_decoder_tokens, self.embedding_dim, name = "decoder_embedding")
    self.embedding_results2 = self.embedding_layer2(self.decoder_inputs,)

    self.decoder = self.get_decoder( self.embedding_dim, self.type_decoder_unit, self.num_of_decoder_layer,)
    self.decoder_results = self.decoder(self.embedding_results2, initial_state=self.encoder_states)

    self.decoder_output = self.decoder_results[0]
    self.decoder_dense = self.get_dense_layer(self.num_decoder_tokens)
    self.dense_output = self.decoder_dense(self.decoder_output)

  def build_model(self):
    
    self.model = keras.Model([self.encoder_inputs, self.decoder_inputs], self.dense_output, name = "Seq2Seq_model")
    return self.model



In [252]:

seq2seq = Seq2seq(num_encoder_tokens,num_decoder_tokens, 1024,1,"rnn", 0.0, 0.0).build_model()
seq2seq.summary()

(None, None, 1024)
Model: "Seq2Seq_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_layer_1 (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Input_layer_2 (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 1024)   26624       Input_layer_1[0][0]              
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, None, 1024)   66560       Input_layer_2[0][0]              
___________________________________________________________________

## Train the model


In [182]:
seq2seq.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')]
)



In [270]:
class BeamSearch(keras.callbacks.Callback):

  def __init__(self, beam_size):
    self.beam_size = beam_size

  def beam_search_decoder(aelf, data, k):
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for row in data:
      all_candidates = list()
      # expand each current candidate
      for i in range(len(sequences)):
        seq, score = sequences[i]
        for j in range(len(row)):
          candidate = [seq + [j], score - log(row[j])]
          all_candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      # select k best
      sequences = ordered[:k]
    return sequences
  
  def on_epoch_end(self, epoch, logs = None):
    prediction = self.model.predict([val_enc_input_data , val_dec_input_data])
    print(prediction.shape)
    for i, pred in enumerate(prediction):
      beam_search_prediction = self.beam_search_decoder(pred, self.beam_size)
      correct_prediction = 0
      for k in range(self.beam_size):
        #translated_word = "\t"+"".join([reverse_target_token[x] for x in beam_search_prediction[k][0][:len(validation_target[i])-1]])
        #print(translated_word, validation_target[i])
        #print(validation_target[i])
        
        def idx2char(idx_list):
          return "".join([reverse_target_token[x] for x in idx_list])

        if "\t"+ idx2char(beam_search_prediction[k][0][:len(validation_target[i])-1]) == validation_target[i]:
          correct_prediction+=1
          break
    mul = 10.0**2
    logs["character_accuracy"] = ((correct_prediction/prediction.shape[0])*mul)/mul
    print("- character_accuracy",logs["character_accuracy"])
    #print(f"Accuracy by Beam Search {correct_prediction/len(validation_target)}")
      # print(len(beam_search_prediction))
      # print(beam_search_prediction)


In [289]:
def beam_search_decoder(data, k):
    decodedWords = [[list(), 0.0]]
    # walk over each step in sequence
    for word in data:
      candidates = list()
      # expand each current candidate
      for sequence in decodedWords:
        seq, score = sequence
        for j in range(len(word)):
          candidate = [seq + [j], score - log(word[j])]
          candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(candidates, key=lambda a:a[1])
      # select k best
      decodedWords = ordered[:k]
    return decodedWords
  
def translate(seq):
  sentence = [] 
  for x in seq:
    char = reverse_target_token[x]
    sentence.append(char)
  return "".join(sentence)
class WordAccuracyCallback(keras.callbacks.Callback):
  def __init__(self,beam_size):
    self.beam_size=beam_size
  def on_epoch_end(self, epoch, logs=None):
    pred=self.model.predict([val_enc_input_data , val_dec_input_data])
    count=0
    for i in range(pred.shape[0]):
      pSequences=beam_search_decoder(pred[i],self.beam_size)
      for j in range(self.beam_size):
        if "\t"+translate(pSequences[j][0][:len(validation_target[i])-1])==validation_target[i]:
          count=count+1
          break
    factor = 10.0 ** 4
    logs["WordAccuracy"]=math.trunc((count/pred.shape[0])*factor)/factor
    print("- wordAccuracy:",logs["WordAccuracy"])

In [185]:
sweep_config = {
    
    'method':'bayes',
    'metric': {
        'name':'val_accuracy',
        'goal':'maximize'
    },
    'parameters':{
    
    "num_of_layer" : {'values': [1,2,3]},
    "unit_size": {"values":[16,32,64]},
    "unit_type": {"values":["lstm","rnn","gru"]},
    "dropout": {"values": [0.0, 0.2, 0.4]},
    'recurrent_dropout':{'values':[0.0,0.3]},
    "beam_size" : {"values":[1,2,3,4]},
    "epochs":{"value":20},  
    "optimizer":{"values": ["adam","rmsprop"]}             
                   }
}



pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'beam_size': {'values': [1, 2, 3, 4]},
                'dropout': {'values': [0.0, 0.2, 0.4]},
                'epochs': {'value': 20},
                'num_of_layer': {'values': [1, 2, 3]},
                'optimizer': {'values': ['adam', 'rmsprop']},
                'recurrent_dropout': {'values': [0.0, 0.3]},
                'unit_size': {'values': [16, 32, 64]},
                'unit_type': {'values': ['lstm', 'rnn', 'gru']}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="seq2seq")

In [ ]:
def train(config = None):
  with wandb.init(config=config):
    config = wandb.config
    #print(config)
    seq2seq = Seq2seq(num_encoder_tokens,num_decoder_tokens, config.unit_size, config.num_of_layer,config.unit_type , config.dropout,config.recurrent_dropout).build_model()
    seq2seq.compile(optimizer=config.optimizer, loss="categorical_crossentropy", metrics=["accuracy",])
    seq2seq.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=config.epochs,
        validation_data =  ([validation_encoder_input_data , validation_decoder_input_data] ,validation_decoder_target_data),
        callbacks = [BeamSearch(config.beam_size), WandbCallback()],verbose = 1, 
        )


    
    
wandb.agent(sweep_id, train)

In [290]:
seq2seq.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')]
)
seq2seq.metrics_names



[]

In [312]:
pred=seq2seq.predict([val_enc_input_data , val_dec_input_data])
count=0
for i in range(pred.shape[0]//400):
      pSequences=beam_search_decoder(pred[i],3)
      for j in range(3):
        print({"\t"+translate(pSequences[j][0][:len(validation_target[i])-1])}, "original =", {validation_target[i]} )
        if "\t"+translate(pSequences[j][0][:len(validation_target[i])-1])==validation_target[i]:
          count=count+1
          print("yes")
          break
factor = 10.0 ** 4


{'\tकं\nा'} original = {'\tअंक\n'}
{'\tवं\nा'} original = {'\tअंक\n'}
{'\t्ं\nा'} original = {'\tअंक\n'}
{'\tकं\nा'} original = {'\tअंक\n'}
{'\tवं\nा'} original = {'\tअंक\n'}
{'\t्ं\nा'} original = {'\tअंक\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकित\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकित\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकित\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकों\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकों\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकों\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकों\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकों\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकों\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकों\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकों\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकों\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tकन\n\n\n\n\n'} original = {'\t

In [301]:
x = seq2seq.predict([val_enc_input_data , val_dec_input_data])
x.shape

(4502, 17, 65)

In [291]:

histotry = seq2seq.fit(
    [enc_input_data, dec_input_data],
    dec_target_data,
    batch_size=8192,
    epochs=1,
    callbacks = [WordAccuracyCallback(3), ],
)
# Save model
seq2seq.save("s2s")


6/6 [==============================] - 214s 34s/step - loss: 1.1271 - acc: 0.1550
- wordAccuracy: 0.0


In [ ]:
# for key in histotry.history.keys():
#       print(key , histotry.history[key])
#       #wandb.log({key : histotry.history[key]})

In [ ]:
# seq2seq.metrics_names

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:
# # Define sampling models
# # Restore the model and construct the encoder and decoder.
# model = keras.models.load_model("s2s")

# encoder_inputs = model.input[0]  # input_1
# temp = model.layers[2].output
# encoder_outputs, state = temp[0], temp[1:]  # lstm_1
# encoder_states = state
# encoder_model = keras.Model(encoder_inputs, encoder_states)

# decoder_inputs = model.input[1]  # input_2
# decoder_state_input_h = keras.Input(shape=(latent_dim,))
# decoder_state_input_c = keras.Input(shape=(latent_dim,))
# decoder_states_inputs = state
# decoder_lstm = model.layers[3]
# temp = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# decoder_outputs, state_dec = temp[0], temp[1:]
# decoder_states = state_dec
# decoder_dense = model.layers[4]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = keras.Model(
#     [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
# )

# # Reverse-lookup token index to decode sequences back to
# # something readable.
# # reverse_input_char_index = dict((i, char) for char, i in num_encoder_tokens.items())
# # reverse_target_char_index = dict((i, char) for char, i in num_decoder_tokens.items())
# # print(reverse_input_char_index)
# # print(input_token_index)

# reverse_input_token = dict((i, char) for char, i in input_token.items())
# reverse_target_token = dict((i, char) for char, i in target_token.items())
# def decode_sequence(input_seq):
#     # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1.
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     # Populate the first character of target sequence with the start character.
#     target_seq[0, 0, target_token_index["\t"]] = 1.0

#     # Sampling loop for a batch of sequences
#     # (to simplify, here we assume a batch of size 1).
#     stop_condition = False
#     decoded_sentence = ""
#     while not stop_condition:
#         temp = decoder_model.predict([target_seq] + states_value)
#         output_tokens, state = temp[0],temp[1:]

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         #print(reverse_target_char_index)
#         sampled_char = reverse_target_token[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length
#         # or find stop character.
#         if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.0

#         # Update states
#         states_value = state
#     return decoded_sentence



You can now generate decoded sentences as such:


In [ ]:
# for seq_index in range(20):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index : seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print("-")
#     print("Input sentence:", input_texts[seq_index])
#     print("Decoded sentence:", decoded_sentence)


In [314]:
! git log

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
